In [1]:
%pip install --upgrade pip
%pip install numpy pandas
%pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


# 분류기(Classifier) 학습하기

## 이미지 분류기 학습하기

### 학습 단계

1. `torch.vision`을 사용하여 CIFAR10의 학습용 / 시험용 데이터셋을 불러오고, 정규화(nomarlizing) 한다.
2. 합성곱 신경망(convolution Neural Network)를 정의한다.
3. 손실 함수를 정의한다.
4. 학습용 데이터를 사용하여 신경망을 학습한다.
5. 시험용 데이터를 사용하여 신경망을 검사한다.

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

## 1. CIFAR10을 불러오고 정규화하기

In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        (0.5, 0.5, 0.5),
        (0.5, 0.5, 0.5)
    )
])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

testset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

classes = (
    'plane',
    'car',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
import numpy as np
import matplotlib.pyplot as plt

def imshow(image):
    image = image / 2 + 0.5
    plt.imshow(np.transpose(image.numpy(), (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print(' '.join(f"{classes[labels[i]]:5s}" for i in range(batch_size)))

bird  dog   truck dog  


## 2. 합성곱 신경망(Convolution Neural Network) 정의하기

In [16]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 *5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
    
net = Net()

# 3. 손실 함수와 Optimizer 정의하기

교차 엔트로피 손실(cross entropy loss)과 모멘텀(momentum) 값을 갖는 SGD를 사용한다.

In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    net.parameters(),
    lr=1e-3,
    momentum=0.9
)

## 4. 신경망 학습하기

단순히 데이터를 반복해서 신경망에 입력으로 제공하기, 최적화(Optimize)만 하면 된다.

In [18]:
for epoch in range(2):
    running_loss = 0.0
    for index, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if index % 2_000 == 1_999:
            print(f"[{epoch + 1}, {index + 1:5d}] loss: {running_loss / 2_000:.3f}")
            running_loss = 0.0

print("Finished Traning")

[1,  2000] loss: 2.170
[1,  4000] loss: 1.851
[1,  6000] loss: 1.655
[1,  8000] loss: 1.561
[1, 10000] loss: 1.524
[1, 12000] loss: 1.474
[2,  2000] loss: 1.425
[2,  4000] loss: 1.395
[2,  6000] loss: 1.340
[2,  8000] loss: 1.328
[2, 10000] loss: 1.296
[2, 12000] loss: 1.293
Finished Traning


In [19]:
torch.save(
    net.state_dict(),
    './cifar_net.pth'
)

## 5. 시험용 데이터로 신경망 검사하기

신경망이 예측한 출력과 진짜 정답(Ground-truth)을 비교하는 방식으로 확인한다.

In [20]:
dataiter = iter(testloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print("GroundTrutch: ", " ".join(f"{classes[labels[i]]}" for i in range(4)))

GroundTrutch:  cat ship ship plane


In [21]:
net = Net()
net.load_state_dict(torch.load('./cifar_net.pth'))

<All keys matched successfully>

In [22]:
outputs = net(images)

In [23]:
_, predicted = torch.max(outputs, 1)
print("Predicted: ", " ".join(f"{classes[predicted[i]]:5s}" for i in range(4)))

Predicted:  cat   ship  car   plane


In [25]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy ot the network on the 10,000 test images: {100 * correct // total}%")

Accuracy ot the network on the 10,000 test images: 57%


In [28]:
correct_pred = {
    classname: 0 for classname in classes
}
total_pred = {
    classname: 0 for classname in classes
}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f"Accuracy for class: {classname:5s} is {accuracy:.1f}%")

Accuracy for class: plane is 71.0%
Accuracy for class: car   is 67.7%
Accuracy for class: bird  is 36.8%
Accuracy for class: cat   is 22.7%
Accuracy for class: deer  is 49.6%
Accuracy for class: dog   is 60.7%
Accuracy for class: frog  is 65.7%
Accuracy for class: horse is 67.5%
Accuracy for class: ship  is 69.8%
Accuracy for class: truck is 59.6%


In [29]:
del dataiter